# 2.1 理解word embeddings

1.LLM 无法直接处理raw text，因为文本是分类数据，与用于实现神经网络的数学运算是不兼容的。因此我们就需要一种将原始数据转换为向量形式的工具。<br>

2.embedding的概念：将数据（可以是多种格式：text, video, audio）转换为向量(vector)格式。embedding实际上就是将离散的对象转换为连续向量空间中的点的过程，embedding的目的就是将非数值（non-numeric）的数据转换为神经网络可以处理的格式。<br>

3.不同的数据格式，需要的embedding模型是不一样的，比如文本的和视频的就不一样。并且，同样是对于文本嵌入（text embedding），也会存在多种嵌入的类型，比如word embedding, sentence/paragraphs/whole documents embedding等。Sentence/paragraphs embedding 是检索增强生成（retrieval-augmented generation , RAG）比较常用的embedding格式。<br>

Note：<br>
RAG是将生成和检索（搜索外部知识库）结合，用于拉取相关知识然后生成文本的方法。<br>

4.最常用的word embedding工具是Word2Vec方法，实际上就是用预训练好的神经网络来预测给定目标词的上下文或者给定上下文来预测目标词，来生成词嵌入。<br>
在word embedding中，相似概念的词，在词嵌入空间中的距离就离得越近。<br>

5.Word embedding的维度可以从1到上千那么大，但是维度越高一方面可以捕捉更加细微的关系，但是计算资源的消耗也会更大。Embedding size一般指的是模型的隐藏状态(hidden states)的维度。<br>

6.LLM一般不会使用传统的词嵌入模型来生成word embedding，而是有专门的embedding layer，并且是trainable的，主要就是为了能够在具体的任务和数据上进行优化和适配。<br>

# 2.2 Tokenizeing text

1.Token指的是独立的单词或者特殊的字符，也包括标点符号。<br>

2.在Tokenization过程中，通常要避免将text中的所有字母都变成小写了，因为大小写通常会帮助LLM区分专有名词和普通名词、理解句子结构、并且学习如何在输出中生成正确的大小写。<br>

3.在训练tokenizer的时候，是否要将空格也编码为独立的符号，需要根据实际的应用场景来判断。比如在我们对文本的结构十分敏感的时候，就需要保留空格，比如在代码生成任务中的时候。<br>

# 2.3 将Token转换为Token IDs

1.将从text进行tokenization之后得到的tokens转换为数值表示，以生成所谓的 token IDs。这是将token IDs转换为embedding vectors的前序步骤。<br>

2.为了实现这个转换，我们必须构建一个 vocabulary，它定义了如何建每一个独特的单词和符号转换为独特的数值。<br>

3.在构建这个vocabulary之前，通常需要将这些tokens按照字母表的顺序首字母排列，并且去重，然后将这些独特的token映射为独特的数值。<br>

4.上面3讲的这个过程是从token ——》 token IDs，这是LLM输入阶段需要进行的处理。如果是LLM输出的token IDs，我们应该建立一个反向的映射vocabulary，将LLM的输出映射回字符串。<br>

5.一个完整的tokenizer类必须包含两个方法：<br>
（1）encode:将文本划分为tokens，并且实施string-to-integer的映射，通过vocabulary来产生token IDs。<br>
（2）decode：实施integer-to-string的映射，将token IDs转换回文本text。<br>

# 2.4 添加Context Tokens

1.Context Tokens：<br>
（1）<|unk|>：表示新的或者未知的单词，实际上就是训练数据中没有的，并且在字典中没有记录的单词；遇到字典中没有的单词的时候使用。<br>

（2）<|endoftext|>:用于分隔两个不相关的文本，标志着一段文本的结束或者开始；<br>

（3）[BOS](begining of sequence): 标志着一段文本的开始；<br>

（4）[EOS](end of sequence)：放置在文本的末尾，主要用于连接多个不相关的文本的，与<|endoftext|>作用相同；<br>

（5）[PAD](padding)：一般是在LLM的batch sizes大于一的时候，一个batch中有不同长度的text，为了使得所有text的长度一致，就会以最长的text的长度为标准，把长度小于此的text扩展成一样长，而扩张就是使用的[PAD]这个token。<br>
在LLM中，用于扩展（padding）操作的token除了[PAD]以外，也会使用<|endoftext|>，但是实际上LLM会使用一个mask，来掩蔽那些新增的padded token，因此使用什么token来进行padding不重要，因为这些padded token都会被忽略掉。<br>

2.注意：<br>
（1）GPT这些模型除了使用<|endoftext|> token之外，不会使用以上的其他任何token。<br>
（2）GPT使用的tokenizer对于那些不在字典中的单词，也不会使用<|unk|> token，而是使用byte pair encoding (BPE) tokenizer，主要就是将单词(words)划分为子词单元(subword units)。<br>

# 2.5 字节对编码Byte Pair Encoding (BPE)

1.BPE是用来训练GPT-2 GPT-3 和 ChatGPT的tokenizer，使用的是openai的tiktoken库（使用Rust写的）。<br>

2.BPE tokenizer的vocabulary size是50257，其中<|endoftext|> token的token ID是50256，也就是最后一个。<br>

3.BPE能够处理任何未知的单词，原理是：<br>
（1）BPE构建vocabulary的方式：是通过迭代地将频繁出现的字符(characters)合并为子词（subwords），然后再将频繁出现的子词合并为完整的单词（words）来构建的。<br>

（2）BPE处理未知单词的方式：将单词划分为在预先定义的词典中已经有的，更小的字词或者独立的字符。这样使得BPE就能够将未知的单词表示为字词或者字符的序列，而摆脱了对<|unk|> token的依赖。<br>

# 2.6 使用滑动窗口进行数据采样

1.在LLM的输入格式是输入-目标对（input-target pair），在训练过程中，会将target之后的所有单词都给掩码（mask）掉，使得模型只能根据target之前的单词进行预测。

2.具体来讲，在next token prediciton任务中，input-target pair的格式是：

（1）input是一个tensor，每一行代表着一个输入的内容（context），也就是sample。

（2）target也是一个tensor，包含了要预测的下一个token，一般是将在input的索引基础上后移（shift）1个单位得到的。

3.一般是使用pytorch的Dataset和Dataloader类来创建生成batched input-target对的dataloader。并且对于一些构造时参数的选择的说明如下:

（1）对于LLM的输入，input中的一个sample的max_length，也就是包含的token数量，一般至少是256个tokens。

（2）对于stride的设置，stride代表的是在batch之间，input移动的位置数量，可以理解为滑动窗口（sliding window）。<br>
如果将stride设置为和输入的window size（就是max_length）一样大，那么batch之间就不会有重叠，这样做的好处是既能够不跳过任何一个词（因为stride > max length的话，就会有token被跳过），而且能够避免因为batch之间过多的重叠导致的过拟合问题。

注意：这里batch实际上指的是一个batch中的一个sample。




# 2.7 创建token embedddings

1.将输入的text转换为LLM的输入一共包括三步：<br>
tokenizing text (tokenization) ——> converting text tokens to token IDs ——> converting token IDs into vector embedding vectors (embedding).<br>

2.在embedding操作之前，需要先随机初始化embedding layer 的权重矩阵，这标志着LLM训练的开始，并且这些权重在训练过程中是不断更新优化的。<br>

3.embedding操作的具体实现：<br>
```python
    embedding_layer = torch.nn.Embedding(vocab_size, embedding_size)
```
（1）GPT-3的embedding size = 12288；<br>
（2）对于embedding层的权重矩阵embedding_layer.weight: 它的shape是(vocab ,embedding_size),每一行代表着vocabulary中的一个token，每一列代表着embedding的其中一个维度。<br>

4.embedding操作的理解：<br>
（1）embedding laye实际上是一种查找操作（look-up），就是根据token的token ID，到embedding layer的权重矩阵中去检索第token ID + 1行，将这一行形状为（1, embedding）的值作为该token的embedding结果。<br>
这里检索第token ID + 1行的原因是token ID是从0开始编号的，token ID = 10，那么在embedding layer中对应的是第11行的值。<br>

（2）embedding layer实际上是一种先实现one-hot编码，然后在全连接层中进行矩阵乘法运算的高效实现方式。并且embedding layer是可以通过神经网络的反向传播算法进行优化更新的。<br>

5.embedding操作的其他实现方式:one hot 编码 + 矩阵乘法。<br>
具体的实现方式参见[code](https://github.com/rasbt/LLMs-fromscratch/tree/main/ch02/03_bonus_embedding-vs-matmul)<br>

这里面主要分为两个步骤：<br>
（1）one hot encoding: 使用`one_hot = torch.nn.functional.ont_hot(torch.tensor([token_ids]))`得到token IDs的one hot 编码<br>

（2）matrix multiplication: 使用`linear = torch.Linear(vocab_size, embedding_size)`得到embedding layer（这里和使用torch.nn.Embedding的不一样），然后通过 `embedding = linear(one_hot.float())` 得到最终的embedding结果。<br>

这种方式相较于直接使用torhc.nn.Embedding的方式来讲，结果是一样的，但是这种方式的效率更低，主要是因为矩阵相乘的时候会有很多乘以0的冗余操作。<br>

# 2.8 编码单词的位置

1.需要位置编码的原因：
（1）LLM的self-attention机制对于一个序列中token的位置或者顺序没有概念；并且embedding layer也没有考虑token在序列中的位置信息。<br>

（2）虽然确定性（deterministic）和与位置无关（position-independent）的token ID的embedding有利于可重复性（因为不同位置的相同token ID的embedding是一样的，使得模型的随机性会减少）。<br>
而且LLM的自注意力机制也是位置无关(position-agnostic)的，但是模型为了具备顺序感知能力，比如“狗咬人”和“人咬狗”是不一样的，所以向embedding中加入位置信息是有帮助的。<br>

2.位置编码主要有两种：<br>
（1）绝对位置编码absolute postional embedding:直接将token在序列中的绝对位置编码进入token的embedding中。注意：positional embedding和token embedding的维度是一样的。<br>

（2）相对位置编码relative positional embedding:将token的相对位置或者token之间的距离编码进入token embedding，意味着模型学习的是“有多远”，而不是“具体位置在哪儿”，这种方法的优势是模型即使在训练的时候没有见过的长度上，仍然可以泛化得很好。<br>

3.位置编码的主要作用是为了增强LLM理解token之间的顺序和关系的能力，使得模型输出更加精确和上下文感知（context-aware）的输出。<br>
OpenAI的GPT系列的模型使用的是绝对位置编码，并且在训练过程中是会被优化的，这与原始的Transformer中固定的位置编码不同。<br>

4.位置编码的具体实现
```python
    pos_embedding_layer = torch.nn.Embedding(context_length, embedding_size)
    pos_embeddings = pos_embedding_layer(torch.arange(context_length))

    # Note:这里context_length就是输入的input text sequence的长度，一般是数据集里面的max_length参数决定的。
```